In [2]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [21]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    keys_list = []
    for _ in range(n):
        x_temp, y_temp = generate_keys(g, p)
        keys_list.append([x_temp, y_temp])
    return keys_list


def concat_and_shuffle_total_keys(keys_list, x, y):
    keys_list.append([x, y])
    random.shuffle(keys_list)
    return keys_list


def get_real_key_position(keys_list, y):
    public_key_list = [key[1] for key in keys_list]
    return public_key_list.index(y)


def get_Y(keys_list):
    public_key_list = [key[1] for key in keys_list]
    Y = ''.join([str(i) for i in public_key_list])
    return Y


def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R


def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    e_list = [None for _ in range(len(keys_list))]
    sig_list = [None for _ in range(len(keys_list))]
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    y_wave = pow(h, x_real, p)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    e_list[idx] = e_initial
    while keys_list[idx][1] != y_real:
        e_last_one = e_list[idx]
        _, random_sig = generate_keys(g, p)
        sig_list[idx] = random_sig
        temp_y = keys_list[idx][1]
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        idx = (idx + 1) % len(total_keys_list)
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
        'e_list': e_list
    }
    return total_res
        
    
    

# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 10
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
print(total_keys_list)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



[[717578558, 589638606], [6997527, 404602922], [287838987, 874602667], [505951043, 535352816], [648104167, 713565040], [701059670, 797649607], [697144776, 90399850], [733516609, 415995827], [301551036, 385803947], [546808059, 731562744], [150064973, 130725809]]


{'e1': 113666322692879274194121513570641901351491724932382236339517170524601907643909,
 'sig_list': [987619208,
  812033478,
  701374923,
  143466923,
  683376442,
  325704931,
  278024314,
  37458901,
  987166883,
  146437988,
  637520623],
 'public_key_list': [589638606,
  404602922,
  874602667,
  535352816,
  713565040,
  797649607,
  90399850,
  415995827,
  385803947,
  731562744,
  130725809],
 'tag': 614543451,
 'msg': 'Hello, Ring Signature',
 'e_list': [113666322692879274194121513570641901351491724932382236339517170524601907643909,
  1471236104095872006362337891128361459765810056176844130655298971844264860518,
  110198682025433968684473251687151876222038044367346677271189836478545669858396,
  108832303544190409310194781683103768747241582699890550935488572532190141808217,
  28843635767739770331000232150655182231681415992666636028444346447574752939728,
  62158541800723980945918436333631816849041969516428510889981099952998457079798,
  16894877221265311277668387362723690927057787

In [22]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 113666322692879274194121513570641901351491724932382236339517170524601907643909
temp_e_V: 1471236104095872006362337891128361459765810056176844130655298971844264860518
temp_e_V: 110198682025433968684473251687151876222038044367346677271189836478545669858396
temp_e_V: 108832303544190409310194781683103768747241582699890550935488572532190141808217
temp_e_V: 28843635767739770331000232150655182231681415992666636028444346447574752939728
temp_e_V: 62158541800723980945918436333631816849041969516428510889981099952998457079798
temp_e_V: 16894877221265311277668387362723690927057787198512806157146508838467562124762
temp_e_V: 106200733953429833975087786318237210652737126791640619860629297674101410897668
temp_e_V: 45188081695737408671708433348160501832390359074377140586041563092851007081365
temp_e_V: 100228825651007121094054033029610941765072577907916216929425686389759026029548
temp_e_V: 106330473403897097978318697963527659287650661180059018630076061804164495864884
113666322692879274194121513

In [23]:
msg_1 = 'new test'

total_res = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
total_res

{'e1': 84851385026375041411903178072112688468562062939124361182594576342451772497683,
 'sig_list': [751643157,
  727101423,
  187804112,
  90407639,
  505832464,
  700402602,
  146526073,
  513842535,
  80186025,
  409661517,
  150204202],
 'public_key_list': [589638606,
  404602922,
  874602667,
  535352816,
  713565040,
  797649607,
  90399850,
  415995827,
  385803947,
  731562744,
  130725809],
 'tag': 614543451,
 'msg': 'new test',
 'e_list': [84851385026375041411903178072112688468562062939124361182594576342451772497683,
  71330222054994673708101804785576062420500535863653890220331203859313951108733,
  10228737875987893998723372512537673842056256648851755279406110173702929701943,
  69917278317887634347702907515221976541815788974182217444659459062365124093583,
  31596073415828515663520606740273593011467533159290073778576989963115298435444,
  81011410066496008452758019631578653032458242967772440144897270270100734429923,
  8291193563745339530410576235915369571694315146508277855863130

In [24]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True
